In [ ]:
seed = 31

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = Warning)

import numpy as np
from collections import Counter
np.random.seed(seed)

import logging

import random
random.seed(seed)

#tf.keras.mixed_precision.set_global_policy('mixed_float16')
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

#tf.keras.mixed_precision.set_global_policy('mixed_float16')

import keras
from keras.layers import Activation
from keras.utils import get_custom_objects
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
import time
import os

import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

In [ ]:
# prompt: connect to gdrive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:

# Dictionary to store closing prices for each stock
closing_prices = {}

# Path to the directory containing S&P 500 CSV files
sp500_dir = '/content/drive/My Drive/csv'

# Iterate through CSV files in the directory
for filename in os.listdir(sp500_dir):
    if filename.endswith('.csv'):
        ticker = filename.split('.')[0]
        file_path = os.path.join(sp500_dir, filename)

        try:
            # Read CSV file
            df = pd.read_csv(file_path)

            # Check if 'Close' column exists
            if 'Close' not in df.columns:
                print(f"Error: 'Close' column not found in {filename}")
                print(f"Available columns: {df.columns.tolist()}")
                continue

            # Extract closing prices and convert to numpy array
            closing_data = df['Close'].to_numpy()

            # Check if closing_data is not empty before adding to the dictionary
            if closing_data is not None and len(closing_data) > 0 and not np.isnan(closing_data).any():
                # Save closing data in the dictionary
                closing_prices[ticker] = closing_data
                print(f"Loaded {len(closing_data)} closing prices for {ticker}")
            else:
                print(f"No closing prices loaded for {ticker}")

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

# Print the total number of stocks loaded
print(f"\nTotal number of stocks loaded: {len(closing_prices)}")

# Print a sample of the data to verify
if closing_prices:
    sample_ticker = next(iter(closing_prices))
    print(f"\nSample data for {sample_ticker}:")
    print(closing_prices[sample_ticker][:5])
else:
    print("\nNo data loaded. Check the file paths and CSV format.")


In [ ]:
from sklearn.preprocessing import RobustScaler

def create_sequences(data, seq_length, stride=1):
    x = []
    for i in range(0, len(data) - seq_length + 1, stride):

        x.append(data[i:i+seq_length])
    return np.array(x)

# Sequence length
seq_length = 120
telescope = 7

# Dictionary to store training sequences for each stock
x = []
y = []

for symbol, prices in closing_prices.items():
    if len(prices) >= seq_length:
        x_temp = create_sequences(prices, seq_length, stride = 30)
        x.append(x_temp)
    else:
        print(f"Warning: Not enough data for {symbol}. Needed {seq_length}, but only had {len(prices)}")

x = np.concatenate(x, axis=0)


print(x.shape)



In [ ]:
'''import numpy as np
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm


scaler = RobustScaler()
scaled = scaler.fit_transform(x)

def applyRobustScaler(data):
    elements = []

    # Use tqdm to track the progress of the loop
    for el in tqdm(data, desc="Normalizing Sequences"):
        scaler = RobustScaler()
        reshaped = el.reshape(-1, 1)
        scaled = scaler.fit_transform(reshaped)
        normalized = scaled.flatten()

        elements.append(normalized)

    return np.array(elements)
'''

In [ ]:
has_nan = np.isnan(x).any()

if has_nan:
    print("The array contains NaN values.")
    np.savetxt('array_with_nan_txt.csv', x)
else:
    print("The array does not contain NaN values.")

In [ ]:
from pandas import Series
from sklearn.preprocessing import MinMaxScaler

def applyRobustScaler (data):

  elements = []

  for el in data:

    series = Series(el)
    values = series.values
    values = values.reshape((len(values), 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(values)

    normalized = scaler.transform(values)
    normalized = normalized.flatten()

    elements.append(normalized)

  return np.array(elements)

In [ ]:
def split_xy(data, telescope):
    x = data[:, :-telescope]
    y = data[:, -telescope:]
    return x, y

In [ ]:
from sklearn.preprocessing import RobustScaler

test_size = round(len(x) * 0.1)
#val_size = round(len(x) * 0.2)

train_size = len(x) - test_size

#dataset split
train = applyRobustScaler(x[:train_size])
test = x[train_size:]

#split dataset into data and prediction
x_train, y_train = split_xy(train, telescope)
x_test, y_test = split_xy(test, telescope)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)



In [ ]:
# Reshape the data to (x, y, 1) format
x_train = x_train[:,:, np.newaxis]
y_train = y_train[:,:, np.newaxis]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
input_size = x_train.shape[1:]
output_size = y_train.shape[1]

input_size, output_size

In [ ]:
#model architecture

def buildModel(input_shape, output_shape):

    input_layer = tfkl.Input(shape=input_shape, name='Input')

    x = tfkl.LSTM(128, return_sequences = True, name='lstm')(input_layer)

    cnn = tfkl.Conv1D(128,3,padding = 'same', activation = 'relu')(x)
    cnn = tfkl.MaxPooling1D()(cnn)

    cnn = tfkl.Conv1D(256,3,padding = 'same', activation = 'relu')(cnn)
    cnn = tfkl.MaxPooling1D()(cnn)

    cnn = tfkl.Conv1D(512,3,padding = 'same', activation = 'relu')(cnn)
    gap = tfkl.GlobalAveragePooling1D()(cnn)

    dropout = tfkl.Dropout(.25, seed = seed)(gap)

    dense = tfkl.Dense(512, activation = tf.keras.activations.mish, kernel_constraint=tfk.constraints.MaxNorm(1.5))(dropout)

    dropout = tfkl.Dropout(.1, seed = seed)(dense)

    dense = tfkl.Dense(128, activation = tf.keras.activations.mish, kernel_constraint=tfk.constraints.MaxNorm(1.5))(dropout)

    output_layer = tfkl.Dense(output_shape, activation = 'linear')(dense)

    model = tf.keras.Model(inputs = input_layer, outputs = output_layer, name='LSTMCNN_Model')




    model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        weight_decay=0.004,  # Adam doesn't officially have weight decay in Keras, but you can simulate it
        name="Adam"
    ),
    metrics=[tf.keras.metrics.MeanAbsoluteError()])

    return model

In [ ]:
model = buildModel(input_size, output_size)
model.summary()

In [ ]:
calls = [
            tfk.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 40, restore_best_weights = True),

            tfk.callbacks.ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', patience = 20, factor = 0.4, min_lr = 1e-5)
      ]

In [ ]:
#model training
history = model.fit(

    x = x_train,
    y = y_train,
    batch_size = 128,
    epochs = 300,
    validation_split = .2,
    callbacks = calls

).history

In [ ]:
model.save('model.keras')

TEST MODEL


In [ ]:
from tensorflow.python.keras.models import load_model
model = tf.keras.models.load_model('model.keras')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def predict(model, x):

    #reshape x to fit the model
    x = x[:, np.newaxis]

    #scale x to fit model training
    minmax = MinMaxScaler()
    minmax.fit(x)
    x_scaled = minmax.transform(x)

    # bootstrap predictions 4 times for 7 time steps each, obtaining 28 days prediction
    num_bootstrap_steps = 4
    y_pred_scaled_list = []

    # iteratively predict the next point and append to x_sample_scaled
    current_input = x_scaled.copy()
    for step in range(num_bootstrap_steps):
    # Predict using the model on scaled current input
        y_pred_scaled = model.predict(current_input[np.newaxis, :, :])  # Add batch dimension for prediction

        # Append the predicted value to the list
        y_pred_scaled_list.append(y_pred_scaled[0, :])  # Prediction is 7 points

        # Shift the input sequence by removing the oldest value and appending the new predicted value
        current_input = np.append(current_input[7:], y_pred_scaled.reshape(7,1), axis=0)

    # Inverse transform the predictions to get them back to original scale
    y_pred = minmax.inverse_transform(np.array(y_pred_scaled_list).reshape(-1, 1)).flatten()

    return y_pred

sample_index = random.randrange(0, x_test.shape[0])  # Randomly select an index
x_sample = x_test[sample_index, :]

y_pred = predict(model, x_sample)

y_pred

print(x_sample.shape, y_pred.shape)

# Concatenate the original and predicted values for a smooth connection
all_values = np.concatenate((x_sample, y_pred))

# Create a range for the x-axis (time steps) including the bootstrapped future steps
x_axis = np.arange(x_sample.shape[0] + 4 * 7)

# Create the plot
plt.figure(figsize=(15, 6))

# Plot the input sequence from x_test (scaled)
sns.lineplot(x=x_axis[:x_sample.shape[0]], y=x_sample[:], label="x_test", color='blue')

# Plot the predicted values for the next 7 points, connected to the original sequence
sns.lineplot(x=x_axis[x_sample.shape[0]-1:], y=all_values[x_sample.shape[0]-1:], label="y_test (Predicted)", color='red', marker='o')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import random

# Assuming x_test and y_test are numpy arrays
# Get a sample from x_test and y_test for prediction
sample_index = random.randrange(0, x_test.shape[0])  # Randomly select an index
x_sample = x_test[sample_index, :, np.newaxis]  # Select the test sequence (input for the model)
y_actual = y_test[sample_index, :]  # Ground truth corresponding to the selected sequence

print(x_sample.shape)
# Scale the data using MinMaxScaler
minmax = MinMaxScaler()

# Fit the scaler and transform the x_sample
minmax.fit(x_sample)
x_sample_scaled = minmax.transform(x_sample)

# Bootstrap predictions for the next 7 time steps
num_bootstrap_steps = 4
y_pred_scaled_list = []

# Iteratively predict the next point and append to x_sample_scaled
current_input = x_sample_scaled.copy()
for step in range(num_bootstrap_steps):
    # Predict using the model on scaled current input
    y_pred_scaled = model.predict(current_input[np.newaxis, :, :])  # Add batch dimension for prediction

    # Append the predicted value to the list
    y_pred_scaled_list.append(y_pred_scaled[0, :])  # Prediction is 7 points

    # Shift the input sequence by removing the oldest value and appending the new predicted value
    current_input = np.append(current_input[7:], y_pred_scaled.reshape(7,1), axis=0)

# Inverse transform the predictions to get them back to original scale
y_pred_list = minmax.inverse_transform(np.array(y_pred_scaled_list).reshape(-1, 1)).flatten()

# Create a range for the x-axis (time steps) including the bootstrapped future steps
x_axis = np.arange(x_sample.shape[0] + num_bootstrap_steps * 7)

# Concatenate the original and predicted values for a smooth connection
all_values = np.concatenate((x_sample[:, 0], y_pred_list))
real_values = np.concatenate((x_sample[:, 0], y_actual))

# Create the plot
plt.figure(figsize=(15, 6))
# Plot the input sequence from x_test (scaled)
sns.lineplot(x=x_axis[:x_sample.shape[0]], y=x_sample[:, 0], label="x_test", color='blue')

# Plot the predicted values for the next 7 points, connected to the original sequence
sns.lineplot(x=x_axis[x_sample.shape[0]-1:], y=all_values[x_sample.shape[0]-1:], label="y_test (Predicted)", color='red', marker='o')


#plot the real y
sns.lineplot(x=x_axis[x_sample.shape[0]-1:real_values.shape[0]], y=real_values[x_sample.shape[0]-1:], label="y real", color='green', marker='o')


plt.xlabel("Time Steps")
plt.ylabel("Value")
plt.title(f"Model Predictions (Next month) vs Actual for Sample Index: {sample_index}")
plt.legend()
plt.show()


In [ ]:
#plotting results

best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()


In [ ]:
from tensorflow.python.keras.models import load_model
new_model = tf.keras.models.load_model('model.keras')

In [ ]:
import pickle
filename='model.pkl'

pickle.dump(new_model, open(filename, 'wb'))

In [ ]:
!pip install -U keras

In [ ]:
'''from tqdm.auto import tqdm

# Train model
#####################
num_epochs = 100
hist = np.zeros(num_epochs)

# Number of steps to unroll
seq_dim = seq_length - telescope

for t in tqdm(range(num_epochs), desc="Training Epochs"):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    #model.hidden = model.init_hidden()

    # Forward pass
    y_train_pred = model(x_train)

    loss = loss_fn(y_train_pred, y_train)
    if t % 10 == 0 and t !=0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

In [ ]:
'''#check for GPU support
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
'''def buildModel(input_shape, output_shape):
    model = LSTM(input_shape, output_shape).to(device)

    loss_fn = nn.MSELoss(reduction='mean')

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    return model, optimizer, loss_fn


input_size = x_train.shape[1:]
output_size = y_train.shape[1]



print(input_size, output_size)


num_layers = 3  # Increased number of layers
hidden_size = 128  # Increased number of hidden units
dropout = 0.2


model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout).to(device)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=1e-3)

batch_size = 32
num_epochs = 200


train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(x_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
'''

In [ ]:
'''from tqdm.auto import tqdm
np.set_printoptions(threshold=np.inf)
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    # Removed early stopping and scheduler
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        with open('training_log.txt', 'w') as log_file:
            train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]", leave=True)
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                # Forward pass
                outputs = model(inputs)
                labels = labels.reshape(outputs.shape)
                loss = criterion(outputs, labels)

                # Backward pass and optimization
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

                train_bar.set_postfix({'loss': loss.item()})
                train_bar.update()
        # Validation step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                labels = labels.reshape(outputs.shape)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}')

    return model


In [ ]:
'''from torch.utils.data import DataLoader, TensorDataset
input_size = x_train.shape[1:]
output_size = y_train.shape[1]

print(input_size, output_size)
batch_size = 32
num_epochs = 100


# Assuming x_train, y_train are numpy arrays
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# Create TensorDataset and DataLoader for training
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Assuming you have validation data x_val and y_val
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Create TensorDataset and DataLoader for validation
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Build the model, optimizer, and loss function
model, optimizer, loss_fn = buildModel(input_size, output_size)

# Move model to device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
trained_model = train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device)


In [ ]:
'''import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming x_train and y_train are numpy arrays and have compatible dimensions
# Sampling 100 sequences
num_sequences = 100
sample_indices = np.random.choice(x_train.shape[0], num_sequences, replace=False)

# Plot the first sampled sequence (from the 100)
first_sample_index = sample_indices[0]
first_x_sequence = x_train[first_sample_index, :, 0]  # Select the first feature (if multidimensional)
first_y_sequence = y_train[first_sample_index, :, 0]  # Select the first feature (if multidimensional)

# Create a range for the x-axis (time steps)
x_axis = np.arange(len(first_x_sequence))

# Create the plot
plt.figure(figsize=(15, 6))
ax = sns.lineplot(x=x_axis, y=first_x_sequence, label="x_train", color='blue')

# Highlight the y_train sequence
ax = sns.lineplot(x=[len(first_x_sequence)], y=first_y_sequence, label="y_train", color='red', marker='o')

plt.xlabel("Time Steps")
plt.ylabel("Value")
plt.title("Sampled Sequence of x_train and y_train (Sample Index: {})".format(first_sample_index))
plt.legend()
plt.show()
